In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier

#from sklearn.model_selection import RepeatedStratifiedKFold, GridSearchCV
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score,accuracy_score

Xtrain = pd.read_csv('data/X_train.csv')
ytrain = pd.read_csv('data/y_train.csv')
#print(train['주구매지점'])
#print(train['주구매상품'])

#df.isnull().sum() nan값 세기
#print(train.isnull().sum())

Xtrain['환불금액']=Xtrain['환불금액'].fillna(0) #nan값 0으로 채우기

#상관관계 분석 np.corrcoef(Xtrain['환불금액'],ytrain['gender'])
#print(np.corrcoef(Xtrain['환불금액'],ytrain['gender'])) #거의 상관없음을 알 수 있다.

#one-hot encoding
Xtrain = pd.concat([Xtrain,pd.get_dummies(Xtrain['주구매지점'])],axis=1)
Xtrain = pd.concat([Xtrain,pd.get_dummies(Xtrain['주구매상품'])],axis=1)

Xtrain=Xtrain.drop(['주구매지점','주구매상품'],axis=1)
Xtrain=Xtrain.drop(['소형가전'],axis=1) #test에는 소형가전이 없다.
#standardscaler 사용
print(Xtrain.columns)
print(len(Xtrain.columns))
X=Xtrain.columns


scaler = StandardScaler()
#scaler = MinMaxScaler()
scaler.fit(Xtrain)

Xtrain=scaler.transform(Xtrain)
print(ytrain['gender'].value_counts()) #성별이 여자가 2184개, 남자가 1316개로 좀 차이가 있다.
x_train, x_test, y_train, y_test = train_test_split(Xtrain,ytrain,test_size=0.2)

#로지스틱 회귀 소환
model = LogisticRegression()
#model = SVC(random_state=0)
#model=SVC(kernel='sigmoid', C=1.0, random_state=0, probability=True) #probability를 True로 하는 이유는 predict_proba할 때, 걸릴 수 있다.
#model = KNeighborsClassifier(n_neighbors=5) 
#모델 훈련
model.fit(x_train, y_train['gender'])
#print(model.best_score_)
#model.fit(x_train,y_train['gender'])
print("train acc")
print(accuracy_score(model.predict(x_train),y_train['gender']))
print("train roc_auc")
print(roc_auc_score(y_train['gender'],model.predict(x_train)))
print("test acc")
print(accuracy_score(model.predict(x_test),y_test['gender']))
print("test roc_auc")
print(roc_auc_score(y_test['gender'],model.predict(x_test)))

test = pd.read_csv('data/X_test.csv')

test['환불금액']=test['환불금액'].fillna(0)

test = pd.concat([test,pd.get_dummies(test['주구매상품'])],axis=1)
test = pd.concat([test,pd.get_dummies(test['주구매지점'])],axis=1)

test=test.drop(['주구매지점','주구매상품'],axis=1)
testcustid = test['cust_id']
print(test.columns)
print(len(test.columns))
Y=test.columns

scaler = StandardScaler()
#scaler=MinMaxScaler()
scaler.fit(test)

test = scaler.transform(test)
proba=model.predict_proba(test)
print(proba[:,1]) #남자일 경우(1인 경우) -> 2열에 존재하므로

answer = pd.DataFrame(columns=['custid','gender'])
answer['custid']=testcustid
answer['gender']=proba[:,1]

print(answer)
answer.to_csv('2013_ba.csv',index=False)